#### Importing libraries 
In case of ModuleNotFoundError for plotly you may need to install via Anaconda Navigator. <br>
> Install the below via cut-paste (individually) on Anaconda Prompt <br>
- conda install plotly<br>
-  conda install geopandas<br>
-  pip install pyshp==1.2.10<br>

Signup for Plotly API key @ https://plot.ly/api_signup


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
plotly.tools.set_credentials_file(username='D8EH', api_key='sW19SIUQBx7rx5mQidzv')


#### Importing Data

In [2]:
#Eviction
cities_df = pd.read_csv('cities.csv')

#Realtor
RDC_county_df = pd.read_csv('RDC_InventoryCoreMetrics_County.csv')

#### GRC of data

In [30]:
#Overall file line count
print(f'cities_df is {len(cities_df)} lines long')
print(f'RDC_county_df is {len(RDC_county_df)} lines long')

#Head
print('___cities_df head___')
print(cities_df.head(1))
print('___RDC_county_df head___')
print(RDC_county_df.head(1))

#Median Listing Price range
print(f"Range of RDC_county_df['Median Listing Price']: ${RDC_county_df['Median Listing Price'].min()} - ${RDC_county_df['Median Listing Price'].max()} ")

cities_df is 502872 lines long
RDC_county_df is 1001 lines long
___cities_df head___
    GEOID  year    name parent-location  population  poverty-rate  \
0  100100  2000  Abanda         Alabama         NaN           NaN   

   renter-occupied-households  pct-renter-occupied  median-gross-rent  \
0                         NaN                  NaN                NaN   

   median-household-income   ...    pct-nh-pi  pct-multiple  pct-other  \
0                      NaN   ...          NaN           NaN        NaN   

   eviction-filings  evictions  eviction-rate  eviction-filing-rate  low-flag  \
0               NaN        NaN            NaN                   NaN         0   

   imputed  subbed  
0        0       0  

[1 rows x 27 columns]
___RDC_county_df head___
                 Month  CountyFIPS       CountyName  Nielsen Rank Footnote  \
0  2018-11-01 00:00:00      6037.0  Los Angeles, CA           1.0      NaN   

   Median Listing Price  Median Listing Price M/M  Median Listing Pric

#### Parsing data to determine mapping of cities (Eviction) to counties (Realtor) for 20 chosen cities
- Finding exact names of 'cities' in 'cities_df' <br>
- Finding exact names of 'counties' and State+County FIPS ID of interest in 'RDC_county_df'. Note that source table treated FIPS as float, whereas they should all be string. Significance is that there is no decimal and all values should be 5-digits long, including a leading '0' for any values that are 4-digit prior to decimal when shown in float. <br>
- Upon visual sanitization, this output is going to cities_codex.csv, which will later be imported

In [6]:
def city_beg_with (srch_strng, df = cities_df, field = 'name'):
    '''Returns a list of city names from cities_df '''
    print(srch_strng, end = ': ')
    print(list(set(df[f'{field}'].loc[df[f'{field}'].str.startswith(srch_strng, na = False)])))
    return

def county_beg_with (srch_strng, df = RDC_county_df, field1 = 'CountyName', field2 = 'CountyFIPS'):
    '''Returns a list of county names and State+County FIPS ID from RDC_county_df'''
    print(srch_strng, end = ': ')
    county_nm_list = list(set(df[f'{field1}'].loc[df[f'{field1}'].str.startswith(srch_strng, na = False)]))
    county_FIPS_lst = []
    for i in range(len(county_nm_list)):
        county_FIPS_lst.append((county_nm_list[i], (df.loc[df[field1] == county_nm_list[i], field2].iloc[0])))
    print(county_FIPS_lst)
    return

cities_lst = ['San Francisco', 'Portland', 'Denver', 'Las Vegas', 'Phoenix', 'Seattle', 'Cheyenne', 'St. Louis', 'Chicago', 'Miami', 'New York', 'Kansas City', 'Nashville', 'Los Angeles', 'Dallas', 'Minneapolis', 'Boston', 'San Diego', 'Philadelphia', 'Orlando']
counties_lst = ['San F', 'Multn', 'Denve', 'Clark', 'Maric','King', 'Laram', 'St. Lou','Cook', 'Miami', 'Kings', 'Queen', 'New Y', 'Bronx', 'Richm', 'Jacks', 'Clay', 'Platt', 'David', 'Los A', 'Dalla', 'Henne','Suffo', 'San D', 'Phila', 'Orang']

print('___CITIES___\n')
[city_beg_with(x) for x in cities_lst]

print('\n___COUNTIES___\n')
[county_beg_with(x) for x in counties_lst]
pass

___CITIES___

San Fran: ['San Francisco']
Portl: ['Portland']
Denv: ['Denver City', 'Denver']
Las Veg: ['Las Vegas']
Phoenix: ['Phoenix Lake', 'Phoenix Lake-Cedar Ridge', 'Phoenixville', 'Phoenix']
Seattle: ['Seattle Hill-Silver Firs', 'Seattle']
Cheyenne: ['Cheyenne Wells', 'Cheyenne']
St. Louis: ['St. Louis Park', 'St. Louis', 'St. Louisville']
Chicago: ['Chicago Ridge', 'Chicago', 'Chicago Heights']
Miami: ['Miami Lakes', 'Miami Springs', 'Miami Beach', 'Miami Heights', 'Miamitown', 'Miamiville', 'Miamisburg', 'Miami', 'Miami Shores', 'Miami Gardens']
New York: ['New York', 'New York Mills']
Kansas City: ['Kansas City']
Nashville: ['Nashville-Davidson', 'Nashville']
Los Angeles: ['Los Angeles']
Dallas: ['Dallas Center', 'Dallas', 'Dallas City', 'Dallastown']
Minneapolis: ['Minneapolis']
Boston: ['Bostonia', 'Boston Heights', 'Boston']
San Diego: ['San Diego Country Estates', 'San Diego']
Philadelphia: ['Philadelphia']
Orlando: ['Orlando']

___COUNTIES___

San F: [('San Francisco, CA

#### Creating a clean CountyFIPS value

Step1: Gross reality checking (GRC'ing) the head/tail of sorted 'CountyFIPS' column in 'RDC_county_df'.

Problem 1: 'CountyFIPS' column in 'RDC_county_df' shows 1 NaN value in the final row. A NaN value will error further data manipulation.
Clean 1: Drop final row using drop method.

Problem 2: 'CountyFIPS' column in 'RDC_county_df' returns a float such as 6075.0 or 48113.0 whereas choropleth map uses values without decimal. 
Clean 2: Cast to float then string using astype method.

Problem 3: FIPS should be 5-digit string (no decimal or value after decimal), yet 'CountyFIPS' column in 'RDC_county_df' is either 4 or 5 digits. 
Clean 3: Add leading zeroes using zfill method until len = 5

In [3]:
#Creating copy of 'RDC_county_df' for clean mutating
RDC_county_df_cln = RDC_county_df.copy()

#GRC head/tail of sorted CountyFIPS for anomaly & printed for GRC
RDC_county_df_cln.sort_values(by = 'CountyFIPS')
print(RDC_county_df_cln['CountyFIPS'].head())
print(RDC_county_df_cln['CountyFIPS'].tail())

#Drop final row
RDC_county_df_cln.drop(RDC_county_df_cln.tail(1).index, inplace = True)

#Casting 'CountyFIPS' values to int then str
RDC_county_df_cln['CountyFIPS'] = (RDC_county_df_cln['CountyFIPS'].astype('int')).astype('str')

#Add leading zeroes using str.zfill(5) until len = 5
RDC_county_df_cln['CountyFIPS'] = RDC_county_df_cln['CountyFIPS'].str.zfill(5)

0     6037.0
1    17031.0
2    48201.0
3     4013.0
4     6073.0
Name: CountyFIPS, dtype: float64
996     24035.0
997      5005.0
998     13175.0
999     54069.0
1000        NaN
Name: CountyFIPS, dtype: float64


#### Initial Mapping https://plot.ly/python/county-choropleth/

In [3]:
#This is a working template. Don't touch

df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]
endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = df_sample['FIPS'].tolist()
values = df_sample['Unemployment Rate (%)'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='USA by Unemployment %',
    legend_title='% unemployed'
)
py.iplot(fig, filename='choropleth_full_usa')

C:\Users\adamj\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




The draw time for this plot will be slow for clients without much RAM.


C:\Users\adamj\Anaconda3\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [27]:
#This is the playground

# df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')
# df_sample['State FIPS Code'] = df_sample['State FIPS Code'].apply(lambda x: str(x).zfill(2))
# df_sample['County FIPS Code'] = df_sample['County FIPS Code'].apply(lambda x: str(x).zfill(3))
# df_sample['FIPS'] = df_sample['State FIPS Code'] + df_sample['County FIPS Code']

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]   
endpts = list(np.linspace(0, 1700000, len(colorscale) - 1)) # 
fips = RDC_county_df_cln['CountyFIPS'].tolist()   
values = RDC_county_df_cln['Median Listing Price'].tolist() 

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Median Listing Price',
    legend_title='$'
) # 
py.iplot(fig, filename='choropleth_full_usa') 

C:\Users\adamj\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [15]:
print(RDC_county_df_cln[RDC_county_df_cln['CountyName'] == 'Elko, NV'])
print('32007' in RDC_county_df_cln['CountyFIPS'].tolist() )

# print(RDC_county_df_cln[RDC_county_df_cln['CountyName' == 'Washoe, NV']])

                   Month CountyFIPS CountyName  Nielsen Rank Footnote  \
955  2018-11-01 00:00:00      32007   Elko, NV         956.0      NaN   

     Median Listing Price  Median Listing Price M/M  Median Listing Price Y/Y  \
955              289450.0                    0.0393                    0.0722   

     Active Listing Count   Active Listing Count M/M        ...          \
955                  205.0                   -0.0682        ...           

     Pending Listing Count Y/Y  Avg Listing Price  Avg Listing Price M/M  \
955                       -1.0           308284.0                 0.0198   

     Avg Listing Price Y/Y  Total Listing Count  Total Listing Count M/M  \
955                 0.0509                205.0                  -0.0682   

     Total Listing Count Y/Y  Pending Ratio  Pending Ratio M/M  \
955                  -0.0575            0.0                0.0   

     Pending Ratio Y/Y  
955            -0.0046  

[1 rows x 35 columns]
True
